In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 24.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11s-seg.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 19.7M/19.7M [00:00<00:00, 79.8MB/s]


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="woU6vwyRMS3kZjotoeeA")
project = rf.workspace("angelo-maglasang-vuuq3").project("crack_porosity_v2_small")
version = project.version(1)
dataset = version.download("yolov11")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to crack_porosity_v2_small-1 in yolov11:: 100%|██████████| 884/884 [00:00<00:00, 6606.51it/s]


In [ ]:
!yolo task=segment mode=train model=yolo11s-seg.pt data="/content/crack_porosity_v2_small-1/data.yaml" epochs=100 imgsz=640 plots=True batch=16 patience =20

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=/content/crack_porosity_v2_small-1/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

In [ ]:
import os
import cv2
from ultralytics import YOLO

# Load the pre-trained YOLO model (YOLOv8 in this case)
model = YOLO('/content/runs/segment/train3/weights/best.pt')  # Replace with your trained model

# Input folder containing the images to process
input_folder = '/content/crack_porosity_v2_small-1/test/images'  # Replace with the path to the folder containing your images
output_folder = 'CrackSeg'
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

# Define colors for different classes
green_color = (0, 0, 255)  # Green for class 0
red_color = (255, 255, 255)    # Red for class 1

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Check if the file is an image
        image_path = os.path.join(input_folder, filename)
        frame = cv2.imread(image_path)  # Read the image

        if frame is None:
            print(f"Error: Could not read image {image_path}. Skipping...")
            continue

        # Run YOLO model on the image
        results = model(frame)

        # Process the results
        for result in results:
            boxes = result.boxes  # Bounding boxes

            # Loop through detected boxes and labels
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
                label = int(box.cls)  # Object label (class index)
                score = box.conf.item()  # Convert Tensor to float using .item()

                # Check if the confidence score is above the threshold (50%)
                if score < 0.2:
                    continue  # Skip this detection if confidence is less than 50%

                # Set color based on class
                if label == 0:  # Class 0
                    box_color = green_color
                elif label == 1:  # Class 1
                    box_color = red_color
                else:
                    box_color = (255, 255, 255)  # Default color (white) for other classes

                # Get the class name (optional, based on your dataset)
                class_name = model.names[label]

                # Draw the bounding box with the selected color
                cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)

                # Put the label text with the selected color
                cv2.putText(frame, f'{class_name} {score:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)

        # Save the processed image to the output folder
        output_image_path = os.path.join(output_folder, f'processed_{filename}')
        cv2.imwrite(output_image_path, frame)  # Save the image with bounding boxes

        print(f'Processed {filename}, results saved to {output_image_path}')

print('Processing complete.')



0: 640x640 2 cracks, 20.1ms
Speed: 3.2ms preprocess, 20.1ms inference, 529.9ms postprocess per image at shape (1, 3, 640, 640)
Processed 20220312232822_14196_jpg.rf.c7529e0db3ffc209e8ce6017722aa669.jpg, results saved to CrackSeg/processed_20220312232822_14196_jpg.rf.c7529e0db3ffc209e8ce6017722aa669.jpg

0: 640x640 2 cracks, 20.2ms
Speed: 2.5ms preprocess, 20.2ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)
Processed 7752_17_18_jpg.rf.b0ac5e5490f761adebc626b512bf276a.jpg, results saved to CrackSeg/processed_7752_17_18_jpg.rf.b0ac5e5490f761adebc626b512bf276a.jpg

0: 640x640 5 porositys, 20.1ms
Speed: 2.4ms preprocess, 20.1ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
Processed element-part-welded-structure-arc-welded-seam-technological-holes-element-part-welded-structure-arc-214336100_jpg.rf.74172848b03ab4e9902a7b5a79d6abea.jpg, results saved to CrackSeg/processed_element-part-welded-structure-arc-welded-seam-technological-holes-element-part-wel

In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11n.pt')


# Load the model from a PyTorch checkpoint
model = YOLO('/content/yolo11n.pt')

# Export the model to TFLite format
model.export(format="tflite")



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 313MB/s]


Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.00GHz)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from '/content/yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'onnx>=1.12.0', 'onnx2tf>1.17.5,<=1.22.3', 'onnxslim>=0.1.31', 'tflite_support', 'onnxruntime'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 286.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 249.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 370.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 306.3 MB/s eta 0:00:00
   ━━━━━━━

100%|██████████| 1.11M/1.11M [00:00<00:00, 101MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 44.47file/s]



ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.36...
ONNX: export success ✅ 1.8s, saved as '/content/yolo11n.onnx' (10.3 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.22.3...
TensorFlow SavedModel: export success ✅ 46.4s, saved as '/content/yolo11n_saved_model' (26.0 MB)

TensorFlow Lite: starting export with tensorflow 2.17.0...
TensorFlow Lite: export success ✅ 0.0s, saved as '/content/yolo11n_saved_model/yolo11n_float32.tflite' (10.3 MB)

Export complete (48.6s)
Results saved to /content
Predict:         yolo predict task=detect model=/content/yolo11n_saved_model/yolo11n_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=/content/yolo11n_saved_model/yolo11n_float32.tflite imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


'/content/yolo11n_saved_model/yolo11n_float32.tflite'

In [ ]:
import shutil
from google.colab import files

# Step 1: Zip the folder
shutil.make_archive('/content/train3', 'zip', '/content/runs/segment/train3')  # Creates /content/train3.zip


'/content/train3.zip'

In [ ]:

import cv2
import os
from ultralytics import YOLO

# Load the pre-trained YOLO model
model = YOLO('/content/runs/segment/train3/weights/best.pt')
# Path to the image you want to predict cracks on
image_path = '/content/20241101_092638.jpg'  # Replace with your image path
output_folder = 'CrackPredictions'  # Define an output folder for saving results
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

# Read the image
frame = cv2.imread(image_path)

if frame is None:
    print(f"Error: Could not read image {image_path}. Exiting...")
else:
    # Run the YOLO model on the image
    results = model(frame)

    # Process the results
    for result in results:
        boxes = result.boxes  # Extract bounding boxes from the results

        # Loop through detected boxes and labels
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            label = int(box.cls)  # Object label (class index)
            score = box.conf.item()  # Confidence score

            # Check if the confidence score is above the threshold (e.g., 50%)
            if score < 0.5:
                continue  # Skip this detection if confidence is less than 50%

            # Draw the bounding box on the image
            color = (0, 255, 0)  # Green color for boxes
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Put the label text with the score
            class_name = model.names[label]  # Get class name from the model
            cv2.putText(frame, f'{class_name} {score:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Save the processed image to the output folder
    output_image_path = os.path.join(output_folder, f'predicted_{os.path.basename(image_path)}')
    cv2.imwrite(output_image_path, frame)  # Save the image with bounding boxes

    print(f'Predicted cracks in {os.path.basename(image_path)}, results saved to {output_image_path}')

print('Prediction complete.')




0: 640x640 1 crack, 20.2ms
Speed: 4.1ms preprocess, 20.2ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
Predicted cracks in 20241101_092638.jpg, results saved to CrackPredictions/predicted_20241101_092638.jpg
Prediction complete.
